## White Box on MNIST (Table 3)

In [13]:
from tabulate import tabulate
table = []
DEFENSES = ['Feature squeezing',
               'Label smoothing',
               'FGSM Adversrial Training',
               'VAT',
               'Gaussian + BRELU ',
               'Gaussian + RELU',
               'Vanilla CNN'
           ]
for d in range(len(DEFENSES)):
    table.append([DEFENSES[d]])

In [14]:
import numpy as np

import json
import os, sys
from os.path import join, exists

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from config import DATA_PATH

### White box on adversarial examples frm Deepfool for MNIST test

Defences

In [15]:
CLASSIFIERS = [#"cnn/relu/featsqueeze1", #Feature Squeezing
               #"cnn/relu/labsmooth", # Label Smoothing
               #"cnn/relu/adv-trained/fgsm/eps0.30_train.npy", # Adversarial training FGSM eps=0.4
               #"cnn/relu/adv-trained/vat/eps2.10_train.npy",  # Adversarial Training VAT eps=2.1
               "cnn/brelu/gaussian/stdev0.30/pert-insts10"] # Gaussian sigma =0.3 + brelu
               #"cnn/relu/gaussian/stdev0.30/pert-insts10",  # Gaussian sigma =0.3 + relu
               #"cnn/relu"]  # vanilla cnn-relu

In [16]:
INPUT_PATH = join(DATA_PATH, 'classifiers','mnist')
for c in CLASSIFIERS:
    assert exists(join(INPUT_PATH, c, "best-weights.h5"))
    assert exists(join(INPUT_PATH, c, "model.json"))
    assert exists(join(INPUT_PATH, c, "params.json"))

Adversarial Images

In [17]:
WBox_ATTACKS = join(DATA_PATH,"adversarial",'mnist')

ADV_IMAGES = [#"cnn/relu/featsqueeze1", #Feature Squeezing
              # "cnn/relu/labsmooth", # Label Smoothing
              # "cnn/relu/adv-trained/fgsm/eps0.30_train", # Adversarial training FGSM eps=0.4
              # "cnn/relu/adv-trained/vat/eps2.10_train",  # Adversarial Training VAT eps=2.1
               "cnn/brelu/gaussian/stdev0.30/pert-insts10"] # Gaussian sigma =0.3 + brelu
              # "cnn/relu/gaussian/stdev0.30/pert-insts10",  # Gaussian sigma =0.3 + relu
              # "cnn/relu"]  # vanilla cnn-relu
for c in ADV_IMAGES[0:]:
    print(c)
    #assert exists(join(WBox_ATTACKS, c, "fgsm", "eps0.10_test.npy"))
    assert exists(join(WBox_ATTACKS, c, "deepfool", "test.npy"))

cnn/brelu/gaussian/stdev0.30/pert-insts10


In [18]:
# Load Classifiers
from art.classifiers.cnn import CNN
from art.classifiers.utils import load_classifier
from art.utils import load_mnist
import keras.backend as K
import tensorflow as tf

#important for ensuring the model_weights are loaded in the same session as the one where metrics are computed
session = tf.Session()
K.set_session(session)

In [19]:
(X_train,_), (X_test, Y_test) = load_mnist()

### Robustness

In [8]:
from art.metrics import empirical_robustness_df

for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    attack_imgs = np.load(join(WBox_ATTACKS, a, "deepfool", "test.npy"))
    emp_robust = empirical_robustness_df(X_test, attack_imgs, classifier.model, session)

    temp = table[d]
    temp.append(emp_robust)
    table[d] = temp
    print(DEFENSES[d],emp_robust)
'''
cnn/relu/featsqueeze1 0.161807
cnn/relu/labsmooth 0.183515
cnn/relu/adv-trained/fgsm/eps0.30_train.npy 0.132598
cnn/relu/adv-trained/vat/eps2.10_train.npy 0.0876128
cnn/brelu/gaussian/stdev0.30/pert-insts10 0.228296
cnn/relu/gaussian/stdev0.30/pert-insts10 0.0665115
cnn/relu 0.177879
'''

9013 10000
nan
nan
9.22288
Feature squeezing nan


'\ncnn/relu/featsqueeze1 0.161807\ncnn/relu/labsmooth 0.183515\ncnn/relu/adv-trained/fgsm/eps0.30_train.npy 0.132598\ncnn/relu/adv-trained/vat/eps2.10_train.npy 0.0876128\ncnn/brelu/gaussian/stdev0.30/pert-insts10 0.228296\ncnn/relu/gaussian/stdev0.30/pert-insts10 0.0665115\ncnn/relu 0.177879\n'

### Accuracies

In [20]:
# get classifier
for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    attack_imgs = np.load(join(WBox_ATTACKS, a, "deepfool", "test.npy"))
    acc = classifier.evaluate(x_val=attack_imgs,y_val=Y_test,batch_size=100,verbose=0)[1]*100
    
    temp = table[d]
    temp.append(acc)
    table[d] = temp
    print(DEFENSES[d],acc)

Feature squeezing 10.1700000428


### Robustness as measured by proximty of decision boundary to training examples

In [10]:
from art.metrics import nearest_nieghbour_dist_df

params = {"eps_step": 0.05,
        "eps_max": 1.0,
        "clip_min": 0.0,
        "clip_max": 1.0}

for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    attack_imgs = np.load(join(WBox_ATTACKS, a, "deepfool", "test.npy"))
    avg_nn_dist = nearest_nieghbour_dist_df(X_test, attack_imgs,classifier.model, X_train,  session)
    
    temp = table[d]
    temp.append(avg_nn_dist)
    table[d] = temp
    print(DEFENSES[d],avg_nn_dist)
    


'''
cnn/relu/featsqueeze1 10.3774
cnn/relu/labsmooth 10.3338
cnn/relu/adv-trained/fgsm/eps0.30_train.npy 3.93939
cnn/relu/adv-trained/vat/eps2.10_train.npy 5.41566
cnn/brelu/gaussian/stdev0.30/pert-insts10 16.9473
cnn/relu/gaussian/stdev0.30/pert-insts10 10.3566
cnn/relu 10.5697
'''

1
Feature squeezing 2.35806
Label smoothing 2.26037
FGSM Adversrial Training 2.88795
VAT 2.90772
Gaussian + BRELU  nan
Gaussian + RELU 2.76026
Vanilla CNN 2.31171


'\ncnn/relu/featsqueeze1 10.3774\ncnn/relu/labsmooth 10.3338\ncnn/relu/adv-trained/fgsm/eps0.30_train.npy 3.93939\ncnn/relu/adv-trained/vat/eps2.10_train.npy 5.41566\ncnn/brelu/gaussian/stdev0.30/pert-insts10 16.9473\ncnn/relu/gaussian/stdev0.30/pert-insts10 10.3566\ncnn/relu 10.5697\n'

In [11]:
print(tabulate(table,tablefmt="latex"))

\begin{tabular}{lrrr}
\hline
 Feature squeezing        & 97.23 &   0.179859  &   2.35806 \\
 Label smoothing          & 73.13 &   0.147644  &   2.26037 \\
 FGSM Adversrial Training & 98.26 &   0.0264963 &   2.88795 \\
 VAT                      & 98.41 &   0.021176  &   2.90772 \\
 Gaussian + BRELU         &  9.97 & nan         & nan       \\
 Gaussian + RELU          & 97.07 &   0.105449  &   2.76026 \\
 Vanilla CNN              & 79.16 &   0.155866  &   2.31171 \\
\hline
\end{tabular}


In [ ]:
'''
Flips wrt: true  3561  pred  3390
 9888/10000 [============================>.] - ETA: 0sFeature squeezing 35.61
Flips wrt: true  3706  pred  3547
 9888/10000 [============================>.] - ETA: 0sLabel smoothing 37.06
Flips wrt: true  6136  pred  6027
 9888/10000 [============================>.] - ETA: 0sFGSM Adversrial Training 61.36
Flips wrt: true  8865  pred  8791
 9888/10000 [============================>.] - ETA: 0sVAT 88.65
Flips wrt: true  9073  pred  9061
 9888/10000 [============================>.] - ETA: 0sGaussian + BRELU  90.73
Flips wrt: true  9826  pred  9827
 9888/10000 [============================>.] - ETA: 0sGaussian + RELU 98.26
Flips wrt: true  5549  pred  5448
 9856/10000 [============================>.] - ETA: 0sVanilla CNN 55.49
'''

In [21]:
for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    attack_imgs = np.load(join(WBox_ATTACKS, a, "deepfool", "test.npy"))

In [22]:
attack_imgs

array([[[[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        ..., 
        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]]],


       [[[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan]

In [23]:
ls -l /dccstor/dlw/data/adversarial_learning/adversarial/mnist/cnn/brelu/gaussian/stdev0.30/pert-insts10/deepfool

total 214400
-rw-r--r-- 1 vzantede users       154 Jul 20 11:37 readme.txt
-rw-r--r-- 1 vzantede users  31360096 Jul 20 14:56 test.npy
-rw-r--r-- 1 vzantede users 188160096 Jul 20 14:56 train.npy


In [24]:
np.load('/dccstor/dlw/data/adversarial_learning/adversarial/mnist/cnn/brelu/gaussian/stdev0.30/pert-insts10/deepfool/test.npy')

array([[[[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        ..., 
        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]]],


       [[[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan],
         [ nan],
         ..., 
         [ nan],
         [ nan],
         [ nan]],

        [[ nan],
         [ nan]